In [14]:
import os
import time
import urllib.request
import pandas as pd

# Load the TSV file
tsv_file_path = './update_tle.tsv'

try:
    sheet_values = pd.read_csv(tsv_file_path, sep='\t')
except FileNotFoundError:
    sheet_values = pd.DataFrame(columns=['tle_0', 'tle_1', 'norad_cat_num'])

# Convert to dictionary for quick lookup
tle_dict = {row['norad_cat_num']: row for _, row in sheet_values.iterrows()}

# Fetch satellite catalog
satcat_url = "https://celestrak.com/pub/satcat.txt" # https://api.optimize.esa.int/data/debris_clouds/satcat.txt
try:
    sat_catalogue = urllib.request.urlopen(satcat_url).read().decode('utf-8').strip().split("\n")
except Exception as e:
    raise ConnectionError(f"Failed to fetch satellite catalog: {e}")

sat_dict = {}
for x in sat_catalogue:
    try:
        sat_dict[x.split()[1]] = x
    except IndexError:
        continue


In [20]:
from spacetrack import SpaceTrackClient

username = 'hello@giacomonanni.info'
password = '8pKSdTg39NJhqWU'
st = SpaceTrackClient(username, password)

print(st)

SpaceTrackClient<identity=None>


In [21]:
new_entries = []

for key in sat_dict.keys():
    if str('n_' + key) not in tle_dict:
        try:
            if "-" in sat_dict[key][75:85] and "N/A" not in sat_dict[key][119:127]:
                lines = st.tle_latest(norad_cat_id=[key], format='tle')
                lines = lines.split("\n")

                for line1, line2 in zip(*[iter(lines[-3:-1])]*2):
                    update_line = [str(line1), str(line2), str('n_' + key)]
                    print(update_line)
                    new_entries.append(update_line)

                    # Limit the API request
                    time.sleep(20)
            else:
                continue
        except Exception as e:
            print(f"Error processing satellite {key}: {e}")

# Append new data to the DataFrame and save to TSV
if new_entries:
    new_df = pd.DataFrame(new_entries, columns=['tle_0', 'tle_1', 'norad_cat_num'])
    updated_sheet = pd.concat([sheet_values, new_df])
    updated_sheet.to_csv(tsv_file_path, sep='\t', index=False)
else:
    print("No new entries to update.")

Error processing satellite 00001: 500 Server Error: Internal Server Error for url: https://www.space-track.org/ajaxauth/login
Error processing satellite 00003: 500 Server Error: Internal Server Error for url: https://www.space-track.org/ajaxauth/login
Error processing satellite 00277: 500 Server Error: Internal Server Error for url: https://www.space-track.org/ajaxauth/login
Error processing satellite 00353: 500 Server Error: Internal Server Error for url: https://www.space-track.org/ajaxauth/login
Error processing satellite 00389: 500 Server Error: Internal Server Error for url: https://www.space-track.org/ajaxauth/login
Error processing satellite 00474: 500 Server Error: Internal Server Error for url: https://www.space-track.org/ajaxauth/login
Error processing satellite 00479: 500 Server Error: Internal Server Error for url: https://www.space-track.org/ajaxauth/login
Error processing satellite 00723: 500 Server Error: Internal Server Error for url: https://www.space-track.org/ajaxaut

KeyboardInterrupt: 